## Load GEN

In [ ]:
GEN = keras.models.load_model('train_models/Generator.hdf5')

# MODEL CNN

In [ ]:
Classificator.summary()

## Fit

In [ ]:
# CallBack
#keras.callbacks.EarlyStopping(patience=5),
my_callbacks = [keras.callbacks.ModelCheckpoint(filepath='best_model_CNN.hdf5', save_best_only=True, save_weights_only=True),
                tf.keras.callbacks.CSVLogger('log.scv', separator=",", append=False)]

# Optimizer
opt = keras.optimizers.Adam(learning_rate=0.0001)
# opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compile
Classificator.compile(loss='CategoricalCrossentropy', metrics=['accuracy'], optimizer=opt)

In [ ]:
batch_size = 32
nb_epochs = 20

# Get a train data generator
train_data_gen = data_gen(data=train_data, batch_size=batch_size, Aug=seq, add_labels=add_labels)

# Define the number of training steps
nb_train_steps = 0
for i in range(3):
    nb_train_steps += (add_labels[i] + 1) * cases_count[i]

nb_train_steps //= batch_size
print("Number of training and validation steps: {} and {}".format(nb_train_steps, len(valid_data)))

In [ ]:
# # Fit the model

history = Classificator.fit(train_data_gen, epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                            validation_data=(valid_data, valid_labels), callbacks=my_callbacks)
# class_weight={0:2.0, 1:1, 2:2.0}

## Plots of loss and accuracy

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "bo", label="Потери на этапе обучения")
plt.plot(epochs, val_loss, "b", label="Потери на этапе проверки")
plt.title("Потери на этапах обучения и проверки")
plt.xlabel("Эпохи")
plt.ylabel("Потери")
plt.legend()
plt.show()

plt.clf()
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность на этапах обучения и проверки")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

# Load the best model

In [ ]:
Classificator.load_weights('best_model.hdf5')

# Uploading Train

In [ ]:
# increase Test
test_data = increase(test_data, add_labels=add_labels)

# Preparing test data
if 'seq' not in globals():
    test_data = map(Conv, test_data['image'].values, test_data['label'].values)
else:
    test_data_0 = map(Conv, test_data[test_data['add'] == 0]['image'].values,
                      test_data[test_data['add'] == 0]['label'].values)

    test_data_1 = map(Conv, test_data[test_data['add'] == 1]['image'].values,
                      test_data[test_data['add'] == 1]['label'].values)

    test_data = list(test_data_0)
    for img, label in test_data_1:
        test_data.append((seq(image=img)['image'], label))

test = []
test_labels = []

for i, j in test_data:
    test.append(i.astype(np.float32)/255.)
    test_labels.append(j)

# Convert the list into numpy arrays
test_data = np.array(test)
test_labels = np.array(test_labels)

def shuffle(x, y):
    p = np.random.permutation(len(y))
    return x[p], y[p]

test_data, test_labels = shuffle(test_data, test_labels)

print("Total number of test examples: ", test_data.shape)
print("Total number of labels:", test_labels.shape)

del test, test_data_0, test_data_1

# Test model

In [ ]:
# Evaluation on test dataset
test_loss, test_score = Classificator.evaluate(test_data, test_labels, batch_size=32)

print()

x_0 = 0
x_1 = 0
x_2 = 0
for i in range(test_labels.shape[0]):
    if test_labels[i][0]:
        x_0 += 1
    elif test_labels[i][1]:
        x_1 += 1
    else:
        x_2 += 1

print(x_0, x_1, x_2, '- EXAMPLES')

print()

print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

In [ ]:
# Get predictions
preds = Classificator.predict(test_data, batch_size=16)
preds = np.argmax(preds, axis=-1)

# Original labels
orig_test_labels = np.argmax(test_labels, axis=-1)

print(orig_test_labels.shape)
print(preds.shape)

# ConfusionMatrix and Metrics

In [ ]:
cm  = confusion_matrix(y_true=orig_test_labels, y_pred=preds)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(3), ['Normal', 'Bacteria', 'Viral'], fontsize=16)
plt.yticks(range(3), ['Normal', 'Bacteria', 'Viral'], fontsize=16)
plt.show()

In [ ]:
# Calculate error rate 
accuracy = (cm[0][0] + cm[1][1] + cm[2][2]) / (sum(cm.ravel()))

Recall_0 = cm[0][0] / (cm[0][0] + cm[1][0] + cm[2][0])
Recall_1 = cm[1][1] / (cm[0][1] + cm[1][1] + cm[2][1])
Recall_2 = cm[2][2] / (cm[0][2] + cm[1][2] + cm[2][2])

Precision_0 = cm[0][0] / (cm[0][0] + cm[0][1] + cm[0][2])
Precision_1 = cm[1][1] / (cm[1][0] + cm[1][1] + cm[1][2])
Precision_2 = cm[2][2] / (cm[2][0] + cm[2][1] + cm[2][2])

F_0 = 2 / (1 / Precision_0 + 1 / Recall_0)
F_1 = 2 / (1 / Precision_1 + 1 / Recall_1)
F_2 = 2 / (1 / Precision_2 + 1 / Recall_2)

print("Accuracy of the Classificator is {:.2f}".format(accuracy))

print()

print("Recall of the first class is {:.2f}".format(Recall_0))
print("Recall of the second class is {:.2f}".format(Recall_1))
print("Recall of the third class is {:.2f}".format(Recall_2))

print()

print("Precision of the first class is {:.2f}".format(Precision_0))
print("Precision of the second class is {:.2f}".format(Precision_1))
print("Precision of the third class is {:.2f}".format(Precision_2))

print()

print("F of the first class is {:.2f}".format(F_0))
print("F of the second class is {:.2f}".format(F_1))
print("F of the third class is {:.2f}".format(F_2))